In [2]:
from bs4 import BeautifulSoup
import requests
import re

In [3]:

url =  "https://www.bassettmirror.com/detail.cfm?id=1097/2914-700-926/Symmetry%20Dining%20Table"

In [4]:
req = requests.get(url)

In [76]:
req.status_code

200

In [5]:

soup = BeautifulSoup(req.content, "html.parser")

In [82]:
soup.title.text.strip()

'Bassett Mirror Company'

## Web pages html Code saving for selectors checking

In [6]:
# Saving the prettified HTML to a file
with open("product.html", "w", encoding="utf-8") as file:
    file.write(soup.prettify())

## Nav Bar links scraping

In [8]:
menu = soup.find("ul", id="menu")

In [9]:
collections_link = menu.find_all("a")

In [34]:
menu = soup.find("ul", id="menu")
collections_link = menu.find_all("a")
collections = []
for item in collections_link:
    href = item.get("href")
    if href.startswith("/shop-products.") and not href.endswith("qs=1"):
        collections.append("https://www.bassettmirror.com" + href.replace(" ", "%20"))

In [ ]:
collections

In [ ]:
len(collections)

In [11]:
products = soup.find_all("a", class_ = "col2 span_1_of_3 grid-tn")

In [ ]:
len(products)

## Products list gathering stage

In [ ]:
# Find all <a> tags
links = soup.find_all('a')

products = []

# Iterate through each link
for link in links:
    href = link.get("href")  # Get the href attribute
    if href and href.startswith("/collections/") and "/products/" in href:
        href = "https://parker-house.com" + href
        products.append(href)
        print(href)

In [ ]:
len(products)

## Product page scraping

In [19]:
product_title = soup.find("span", class_ = "pageTitle").text.strip()

In [ ]:
product_title

In [25]:
spec = soup.find('div', class_="spec").get_text().strip()
clean_spec = spec.replace('\xa0', ' ')

In [ ]:
clean_spec

In [ ]:
sku, dim = clean_spec.split("|")

In [32]:
sku = sku.strip()
dim = dim.strip()

In [ ]:
sku

In [ ]:
dim

In [34]:
dims = dim.split("x")

In [35]:
if len(dims)==3:
    width, depth, height = dims
elif len(dims)==2:
    width, height = dims


In [ ]:
width

In [46]:
# Extract the product description
product_description = None  # Initialize with a default value
description_container = soup.find("div", class_="col span_1_of_3")

if description_container:
    description_paragraph = description_container.find("p")
    if description_paragraph:
        product_description = description_paragraph.get_text(strip=True)

In [47]:
product_description

'The Rowan Table Lamp features a cement base  topped with a gold accent and a linen shade. Its modern shape paired with the natural cement finish make this lamp a dynamic addition to any living space.'

In [49]:
specifiation_div = soup.find("div", class_ = "panel" )
specifications = {}
if specifiation_div:
    items = specifiation_div.find_all('li')
    if items:
        for item in items:
            key, value = item.get_text().split(":")
            specifications[key] = value


In [50]:
specifications

{'Color/Finish': 'Gray',
 'Color Details': 'Cement',
 'Material': 'Cement/Hard back linen shade',
 'Style': ' Modern',
 'Shade Material': 'Linen',
 'Shade Color': 'White',
 'Shade Lined': 'Yes',
 'Shade Shape': 'Tapered Drum',
 'Shade Size': '16x17x9',
 'Lamp Base Dimensions': '6x6',
 'Socket Type': 'E26',
 'Switch Type': 'On-Off',
 'Bulb Included': 'No',
 'Bulb Type': 'A',
 'Bulb Wattage': '100 Watt',
 'Cord Length': '60.00"',
 'Harp Size': '7.00"',
 'Harp Color': 'Silver',
 'Shipping Weight': '19.00 lbs',
 'Shipping Method': 'Small Parcel'}

In [51]:
additional_info_div = soup.find('div', class_ = 'spec spec_desc')
additional_info = []
if additional_info_div:
    additional_info_items = additional_info_div.find_all("td")
    if additional_info_items:
        for item in additional_info_items:
            additional_info.append(item.get_text(strip = True))

In [52]:
additional_info

['The beige cement lamp base provides a french country aesthetic',
 'The lamp shade features a slight tapered style, transitioning from a 16 inch diamater at the top to 17 inches at the bot',
 'Features a tapered-drum shaped, beige linen lamp shade',
 'Bulb Type A required, not included',
 'Easy assembly - just attach harp, finial and shade']

In [ ]:

# Scraping image URLs from the gallery
image_urls = []
gallery_div = soup.find("div", id="gallery")
if gallery_div:
    image_divs = gallery_div.find_all("div", class_="tn")
    for div in image_divs:
        style = div.get("style")
        if style:
            # Extract the URL from the background-image style
            url_match = re.search(r'background-image:url\((.*?)\);', style)
            if url_match:
                image_urls.append(url_match.group(1))

product_images = image_urls

In [58]:
product_images

["'https://files.plytix.com/api/v1.1/file/public_files/pim/assets/99/e7/bd/62/62bde79981697600089987d9/images/81/68/e8/63/63e86881002f36906a208fdb/L3223TEC.jpg?s=1000x'",
 "'https://files.plytix.com/api/v1.1/file/public_files/pim/assets/99/e7/bd/62/62bde79981697600089987d9/images/8a/68/e8/63/63e8688a685ae95bd6bea7c1/L3223TEC_B.jpg?s=1000x'",
 "'https://files.plytix.com/api/v1.1/file/public_files/pim/assets/99/e7/bd/62/62bde79981697600089987d9/images/90/68/e8/63/63e86890fae9eef6a69a7c5f/L3223TEC_S1.jpg?s=1000x'",
 "'https://files.plytix.com/api/v1.1/file/public_files/pim/assets/99/e7/bd/62/62bde79981697600089987d9/images/5d/b9/4c/66/664cb95d6f4805fd64550b68/L3223T_S2.jpg?s=1000x'"]

In [61]:
product_images[0]

"'https://files.plytix.com/api/v1.1/file/public_files/pim/assets/99/e7/bd/62/62bde79981697600089987d9/images/81/68/e8/63/63e86881002f36906a208fdb/L3223TEC.jpg?s=1000x'"

In [83]:
paths_div = soup.find("div", class_ = "master-width breadcrumbs")
if paths_div:
    paths_span = paths_div.find("span", class_ = "breadcrumbs")
    if paths_span:
        paths = paths_span.get_text(strip = True).split("/")

In [84]:
paths

['Furniture', 'Dining', 'Tables', 'Symmetry Dining Table']

In [85]:
if len(paths) ==4:
    category1 = paths[0]
    category2 = paths[1]
    category3 = paths[2]
    collection = paths[3]
elif len(paths)==3:
    category1 = paths[0]
    category2 = paths[1]
    collection = paths[2]


In [90]:
category3

'Tables'

In [1]:
st = "Tata bob wwiwiw"

In [2]:
st.split("|")

['Tata bob wwiwiw']

In [7]:
img_url = soup.find("div", id = "main-image")

In [8]:
if img_url:
    img_url = img_url.get("style")

In [9]:
img_url

"background-image:url('https://files.plytix.com/api/v1.1/file/public_files/pim/assets/99/e7/bd/62/62bde79981697600089987d9/images/4c/96/90/64/6490964caeaac3596edb37e5/2914-700-926EC.jpg?s=1000x'); "

In [13]:
img_url = re.search(r'background-image:url\((.*?)\);', img_url)

In [14]:
product_images = img_url.group(1)

In [15]:
product_images

"'https://files.plytix.com/api/v1.1/file/public_files/pim/assets/99/e7/bd/62/62bde79981697600089987d9/images/4c/96/90/64/6490964caeaac3596edb37e5/2914-700-926EC.jpg?s=1000x'"

In [16]:
try:
    specifiation_div = soup.find("div", class_="panel")
    specifications = {}
    if specifiation_div:
        items = specifiation_div.find_all('li')
        if items:
            for item in items:
                key, value = item.get_text().split(":", 1)
                specifications[key.strip()] = value.strip()

    if specifications:
        style = specifications.get("Style", None)
        material = specifications.get("Material", None)
        finish = specifications.get("Color/Finish", None)
        
    # Serialize the specifications dictionary
    specifications_str = "; ".join([f"{k}: {v}" for k, v in specifications.items()])

except AttributeError:
    specifications_str = ""


In [18]:
specifications_str

'Color/Finish: Gold; Color Details: Gold; Material: Hardwood; Style: Modern, Glam/Luxe; Collection: Symmetry; Weight Capacity: 50 lbs; Table Shape: Oval; Seating Capacity: Seats 4-6.; Shipping Weight: 217.00 lbs; Shipping Method: LTL'